## Deep Convolutional Generative Adversarial Network

> This project is referred from the Github repository: https://github.com/eriklindernoren/Keras-GAN/tree/master/dcgan

> The oop programming style is changed into process-oriented programming in ease of jupyter notebook

> Images saved in the folder "images_ipynb"

#### 1. import modules

In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


#### 2. Define the build_generator function, which will return the model (will check out the Model() function later)

In [2]:
def build_generator():

    noise_shape = (100,)
        
    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_shape=noise_shape))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8)) 
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    noise = Input(shape=noise_shape)
    img = model(noise)

    return Model(noise, img)

#### 2. Define the build_discriminator function, which will return the discriminator model 

In [3]:
def build_discriminator():
    img_shape = (img_rows, img_cols, channels)
        
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

#### 3. Define the function that saves images

In [4]:
def save_imgs(epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        #fig.suptitle("DCGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images_ipynb/mnist_%d.png" % epoch)
        plt.close()

#### 4. Define the class DCGAN, with the initialization and training functions

In [5]:

img_rows = 28 
img_cols = 28
channels = 1

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', 
optimizer=optimizer,
metrics=['accuracy'])

# Build and compile the generator
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

# The generator takes noise as input and generated imgs
z = Input(shape=(100,))
img = generator(z)

# For the combined model we will only train the generator
discriminator.trainable = False

# The valid takes generated images as input and determines validity
valid = discriminator(img)

# The combined model  (stacked generator and discriminator) takes
# noise as input => generates images => determines validity 
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    

    

        



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

#### 5. define train

In [6]:
def train(epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                save_imgs(epoch)


#### 5. create a instance for DCGAN, and train the model

In [ ]:
if __name__ == '__main__':
    
    train(epochs=4000, batch_size=32, save_interval=50)

C:\Users\BA0001QI\AppData\Local\Continuum\anaconda3\lib\site-packages\keras\engine\training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.902407, acc.: 46.88%] [G loss: 0.439658]
1 [D loss: 0.763924, acc.: 56.25%] [G loss: 0.535989]
2 [D loss: 0.649352, acc.: 62.50%] [G loss: 0.606408]
3 [D loss: 0.621563, acc.: 71.88%] [G loss: 0.959201]
4 [D loss: 0.490786, acc.: 75.00%] [G loss: 1.102031]
5 [D loss: 0.333417, acc.: 90.62%] [G loss: 1.270628]
6 [D loss: 0.298292, acc.: 90.62%] [G loss: 1.335451]
7 [D loss: 0.308854, acc.: 93.75%] [G loss: 1.370368]
8 [D loss: 0.287669, acc.: 87.50%] [G loss: 1.531201]
9 [D loss: 0.283906, acc.: 87.50%] [G loss: 1.261654]
10 [D loss: 0.312841, acc.: 90.62%] [G loss: 1.593414]
11 [D loss: 0.241797, acc.: 93.75%] [G loss: 2.004383]
12 [D loss: 0.306475, acc.: 93.75%] [G loss: 1.692974]
13 [D loss: 0.179122, acc.: 96.88%] [G loss: 1.737354]
14 [D loss: 0.249898, acc.: 93.75%] [G loss: 1.662880]
15 [D loss: 0.200475, acc.: 93.75%] [G loss: 2.026259]
16 [D loss: 0.211869, acc.: 93.75%] [G loss: 2.296104]
17 [D loss: 0.151197, acc.: 100.00%] [G loss: 2.584612]
18 [D loss: 0.15940

148 [D loss: 0.692429, acc.: 56.25%] [G loss: 1.065437]
149 [D loss: 0.829018, acc.: 53.12%] [G loss: 1.222631]
150 [D loss: 0.998056, acc.: 34.38%] [G loss: 1.278294]
151 [D loss: 1.220851, acc.: 25.00%] [G loss: 1.094982]
152 [D loss: 0.653417, acc.: 62.50%] [G loss: 1.518093]
153 [D loss: 0.725619, acc.: 65.62%] [G loss: 1.725309]
154 [D loss: 0.833992, acc.: 56.25%] [G loss: 1.180690]
155 [D loss: 0.659775, acc.: 68.75%] [G loss: 1.383858]
156 [D loss: 0.635319, acc.: 68.75%] [G loss: 1.309589]
157 [D loss: 0.774112, acc.: 53.12%] [G loss: 1.457096]
158 [D loss: 0.885877, acc.: 43.75%] [G loss: 0.801243]
159 [D loss: 0.683906, acc.: 62.50%] [G loss: 1.176333]
160 [D loss: 1.079083, acc.: 46.88%] [G loss: 1.280458]
161 [D loss: 0.908999, acc.: 50.00%] [G loss: 1.376446]
162 [D loss: 1.032539, acc.: 46.88%] [G loss: 1.265134]
163 [D loss: 0.797673, acc.: 50.00%] [G loss: 1.318629]
164 [D loss: 0.749917, acc.: 56.25%] [G loss: 1.190955]
165 [D loss: 0.786852, acc.: 65.62%] [G loss: 1.

295 [D loss: 0.840636, acc.: 56.25%] [G loss: 1.054120]
296 [D loss: 0.887882, acc.: 46.88%] [G loss: 1.313788]
297 [D loss: 0.627429, acc.: 68.75%] [G loss: 1.076161]
298 [D loss: 0.699562, acc.: 59.38%] [G loss: 1.241869]
299 [D loss: 0.719564, acc.: 62.50%] [G loss: 1.100719]
300 [D loss: 0.770139, acc.: 50.00%] [G loss: 1.270743]
301 [D loss: 0.775158, acc.: 62.50%] [G loss: 0.933501]
302 [D loss: 0.900275, acc.: 53.12%] [G loss: 0.959650]
303 [D loss: 0.796108, acc.: 53.12%] [G loss: 1.282616]
304 [D loss: 0.545692, acc.: 78.12%] [G loss: 1.466231]
305 [D loss: 1.036043, acc.: 31.25%] [G loss: 1.021581]
306 [D loss: 1.005529, acc.: 37.50%] [G loss: 1.208927]
307 [D loss: 0.876571, acc.: 43.75%] [G loss: 1.291313]
308 [D loss: 0.686699, acc.: 53.12%] [G loss: 1.455046]
309 [D loss: 0.740936, acc.: 46.88%] [G loss: 1.313221]
310 [D loss: 0.781486, acc.: 53.12%] [G loss: 1.033411]
311 [D loss: 0.817895, acc.: 59.38%] [G loss: 1.239223]
312 [D loss: 0.707297, acc.: 68.75%] [G loss: 1.

442 [D loss: 0.970323, acc.: 40.62%] [G loss: 1.088381]
443 [D loss: 0.637695, acc.: 68.75%] [G loss: 0.906712]
444 [D loss: 0.813604, acc.: 46.88%] [G loss: 1.083074]
445 [D loss: 0.781277, acc.: 43.75%] [G loss: 1.119287]
446 [D loss: 0.798252, acc.: 53.12%] [G loss: 1.156396]
447 [D loss: 0.725546, acc.: 50.00%] [G loss: 0.835162]
448 [D loss: 0.865333, acc.: 53.12%] [G loss: 0.891121]
449 [D loss: 1.004199, acc.: 28.12%] [G loss: 0.977537]
450 [D loss: 0.573659, acc.: 62.50%] [G loss: 1.625674]
451 [D loss: 0.752980, acc.: 59.38%] [G loss: 1.349520]
452 [D loss: 0.745969, acc.: 53.12%] [G loss: 1.069369]
453 [D loss: 0.676413, acc.: 56.25%] [G loss: 1.048650]
454 [D loss: 0.732593, acc.: 56.25%] [G loss: 0.957101]
455 [D loss: 0.831805, acc.: 46.88%] [G loss: 1.099005]
456 [D loss: 0.727723, acc.: 53.12%] [G loss: 0.959381]
457 [D loss: 0.740233, acc.: 46.88%] [G loss: 0.941880]
458 [D loss: 0.731180, acc.: 59.38%] [G loss: 0.880248]
459 [D loss: 0.860542, acc.: 43.75%] [G loss: 1.

...

1029 [D loss: 0.710681, acc.: 59.38%] [G loss: 0.968496]
1030 [D loss: 0.808363, acc.: 37.50%] [G loss: 0.921353]
1031 [D loss: 0.748870, acc.: 56.25%] [G loss: 1.061141]
1032 [D loss: 0.589093, acc.: 68.75%] [G loss: 1.264704]
1033 [D loss: 0.688039, acc.: 53.12%] [G loss: 1.167070]
1034 [D loss: 0.670684, acc.: 68.75%] [G loss: 1.151988]
1035 [D loss: 0.847104, acc.: 46.88%] [G loss: 0.975869]
1036 [D loss: 0.613431, acc.: 56.25%] [G loss: 0.980566]
1037 [D loss: 0.676505, acc.: 68.75%] [G loss: 0.842162]
1038 [D loss: 0.605374, acc.: 62.50%] [G loss: 1.026921]
1039 [D loss: 0.725711, acc.: 50.00%] [G loss: 1.154741]
1040 [D loss: 0.626918, acc.: 59.38%] [G loss: 1.012131]
1041 [D loss: 0.666154, acc.: 56.25%] [G loss: 0.985498]
1042 [D loss: 0.492538, acc.: 78.12%] [G loss: 1.125892]
1043 [D loss: 0.580642, acc.: 68.75%] [G loss: 1.189940]
1044 [D loss: 0.622464, acc.: 71.88%] [G loss: 1.074337]
1045 [D loss: 0.651704, acc.: 62.50%] [G loss: 1.079228]
1046 [D loss: 0.918623, acc.: 3

1173 [D loss: 0.756988, acc.: 50.00%] [G loss: 1.180692]
1174 [D loss: 0.708275, acc.: 50.00%] [G loss: 1.348074]
1175 [D loss: 0.707757, acc.: 56.25%] [G loss: 1.238021]
1176 [D loss: 0.733870, acc.: 46.88%] [G loss: 1.242535]
1177 [D loss: 0.738641, acc.: 56.25%] [G loss: 1.086692]
1178 [D loss: 0.652766, acc.: 71.88%] [G loss: 1.013622]
1179 [D loss: 0.656256, acc.: 65.62%] [G loss: 1.075988]
1180 [D loss: 0.659312, acc.: 56.25%] [G loss: 1.220856]
1181 [D loss: 0.784482, acc.: 46.88%] [G loss: 0.919588]
1182 [D loss: 0.647260, acc.: 59.38%] [G loss: 1.192776]
1183 [D loss: 0.584063, acc.: 75.00%] [G loss: 1.257545]
1184 [D loss: 0.596215, acc.: 65.62%] [G loss: 0.978793]
1185 [D loss: 0.748425, acc.: 46.88%] [G loss: 1.145097]
1186 [D loss: 0.698461, acc.: 75.00%] [G loss: 1.005114]
1187 [D loss: 0.624546, acc.: 68.75%] [G loss: 1.222661]
1188 [D loss: 0.691765, acc.: 62.50%] [G loss: 1.042812]
1189 [D loss: 0.554215, acc.: 78.12%] [G loss: 1.037189]
1190 [D loss: 0.530829, acc.: 6

1317 [D loss: 0.663708, acc.: 59.38%] [G loss: 1.012346]
1318 [D loss: 0.583135, acc.: 71.88%] [G loss: 0.868022]
1319 [D loss: 0.620618, acc.: 62.50%] [G loss: 0.987913]
1320 [D loss: 0.690259, acc.: 59.38%] [G loss: 1.142942]
1321 [D loss: 0.562759, acc.: 71.88%] [G loss: 0.893162]
1322 [D loss: 0.637482, acc.: 65.62%] [G loss: 1.078066]
1323 [D loss: 0.496242, acc.: 71.88%] [G loss: 1.112717]
1324 [D loss: 0.709205, acc.: 59.38%] [G loss: 0.975736]
1325 [D loss: 0.535588, acc.: 87.50%] [G loss: 1.091727]
1326 [D loss: 0.715416, acc.: 46.88%] [G loss: 1.314260]
1327 [D loss: 0.568211, acc.: 71.88%] [G loss: 1.257182]
1328 [D loss: 0.499668, acc.: 71.88%] [G loss: 1.393700]
1329 [D loss: 0.577776, acc.: 75.00%] [G loss: 1.273915]
1330 [D loss: 0.646627, acc.: 59.38%] [G loss: 1.245475]
1331 [D loss: 0.658556, acc.: 59.38%] [G loss: 1.198068]
1332 [D loss: 0.630085, acc.: 62.50%] [G loss: 0.980621]
1333 [D loss: 0.803600, acc.: 59.38%] [G loss: 1.172945]
1334 [D loss: 0.776702, acc.: 5

1461 [D loss: 0.719984, acc.: 50.00%] [G loss: 0.941196]
1462 [D loss: 0.679999, acc.: 65.62%] [G loss: 1.088877]
1463 [D loss: 0.662072, acc.: 65.62%] [G loss: 1.124354]
1464 [D loss: 0.815345, acc.: 46.88%] [G loss: 1.117188]
1465 [D loss: 0.727572, acc.: 71.88%] [G loss: 1.046385]
1466 [D loss: 0.565830, acc.: 71.88%] [G loss: 0.972116]
1467 [D loss: 0.705380, acc.: 59.38%] [G loss: 1.032201]
1468 [D loss: 0.745711, acc.: 50.00%] [G loss: 1.201559]
1469 [D loss: 0.671414, acc.: 65.62%] [G loss: 1.250131]
1470 [D loss: 0.636559, acc.: 62.50%] [G loss: 1.189504]
1471 [D loss: 0.742401, acc.: 56.25%] [G loss: 1.177509]
1472 [D loss: 0.688032, acc.: 53.12%] [G loss: 1.116601]
1473 [D loss: 0.696512, acc.: 65.62%] [G loss: 1.016150]
1474 [D loss: 0.564718, acc.: 71.88%] [G loss: 1.240003]
1475 [D loss: 0.769575, acc.: 50.00%] [G loss: 0.968294]
1476 [D loss: 0.544344, acc.: 78.12%] [G loss: 0.987717]
1477 [D loss: 0.605820, acc.: 68.75%] [G loss: 0.890529]
1478 [D loss: 0.729276, acc.: 5

1605 [D loss: 0.804124, acc.: 50.00%] [G loss: 1.024778]
1606 [D loss: 0.831486, acc.: 46.88%] [G loss: 1.124093]
1607 [D loss: 0.559210, acc.: 68.75%] [G loss: 1.035478]
1608 [D loss: 0.705123, acc.: 59.38%] [G loss: 1.069409]
1609 [D loss: 0.633014, acc.: 56.25%] [G loss: 1.004148]
1610 [D loss: 0.652551, acc.: 59.38%] [G loss: 0.900982]
1611 [D loss: 0.658405, acc.: 56.25%] [G loss: 1.066358]
1612 [D loss: 0.666789, acc.: 62.50%] [G loss: 1.129076]
1613 [D loss: 0.716405, acc.: 50.00%] [G loss: 1.068013]
1614 [D loss: 0.675274, acc.: 62.50%] [G loss: 0.931964]
1615 [D loss: 0.786124, acc.: 50.00%] [G loss: 1.070537]
1616 [D loss: 0.701361, acc.: 62.50%] [G loss: 1.030697]
1617 [D loss: 0.724866, acc.: 59.38%] [G loss: 1.171723]
1618 [D loss: 0.675537, acc.: 53.12%] [G loss: 1.120259]
1619 [D loss: 0.789362, acc.: 59.38%] [G loss: 0.989810]
1620 [D loss: 0.592753, acc.: 65.62%] [G loss: 1.042818]
1621 [D loss: 0.764827, acc.: 50.00%] [G loss: 1.030154]
1622 [D loss: 0.763425, acc.: 4

#### The output:

<img src = " https://lh5.googleusercontent.com/fkIorDIqqOWoG5r1k2Vhiq2P5tUvZgWSHu7oFrUtpLZAyX6V-Xtqw4nPQCuInCEoHXJV0KBkLoycdTgqeySg=w1884-h1830-rw" >